Hotel Scraping project

In [1]:
!pip install selenium beautifulsoup4 webdriver_manager requests lxml


In [8]:
from typing import *

In [2]:
from datetime import datetime,timedelta

def build_url(time_to_travel: int, length_of_stay: int) -> str:
    today = datetime.today()

    checkin_date = today + timedelta(days=time_to_travel)
    checkout_date = checkin_date + timedelta(days=length_of_stay)
    checkin_date_str = checkin_date.strftime('%Y-%m-%d')
    checkout_date_str = checkout_date.strftime('%Y-%m-%d')
    url = f'https://www.booking.com/searchresults.en-gb.html?ss=New+York&ssne=New+York&ssne_untouched=New+York&lang=en-gb&dest_id=20088325&dest_type=city&checkin={checkin_date_str}&checkout={checkout_date_str}&group_adults=2&no_rooms=1&group_children=0&selected_currency=USD'
    return url

### Interaction functions

#### Genius Modal closer
There is a genius modal that sometimes pops up, and we need to close it

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException


def close_genius_modal(driver: webdriver.Chrome) -> bool:
    try:
        genius_modal_close_button = driver.find_element(By.XPATH, '//button[@aria-label="Dismiss sign in information."]')
        genius_modal_close_button.click()
        return True 
    except (NoSuchElementException, TimeoutException):
        return False

#### Load More 
We are mimiccing a real user interaction by going up and down in the page with the `HOME` and `END` keys. when we cant load more results in this method, we click on the load more results button

In [5]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
def load_more(driver: webdriver.Chrome) -> bool:
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.HOME)
    time.sleep(0.5)
    body.send_keys(Keys.END)

    try:
        # Scroll down to attempt to load more results
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        time.sleep(1)  # Allow new elements to load
        
        # Find the 'Load more results' button
        button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Load more results')]]"))
        )
        
        # Scroll directly to the button before clicking (ensures visibility)
        ActionChains(driver).move_to_element(button).perform()
        button.click()
        
        return True  # Successfully clicked the button

    except Exception:
        return False  # No button found or unable to click


### Mimic a real-user's driver
the Booking.com site can detect if a bot is trying to access it, and it will give it a different type of html, not the one you can see if you try to open the chrome devtools. In order to overcome this problem, and to be able to run the scraping in parrallel, we needed to define the driver in such way, that it will be headless and also behave like a user used site 

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    
    if headless:
        options.add_argument("--headless=new")  # Ensures modern headless mode
        options.add_argument("--disable-gpu")  # Fixes rendering issues
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
    
    # 🚀 Make the browser appear more human-like
    options.add_argument("--window-size=1920,1080")  # Standard screen size
    options.add_argument("--start-maximized")  # Maximize on launch
    options.add_argument("--disable-blink-features=AutomationControlled")  # Prevent detection
    options.add_experimental_option("excludeSwitches", ["enable-automation"])  # Prevent 'bot' flag
    options.add_experimental_option("useAutomationExtension", False)
    
    # Change User-Agent to a normal browser
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Remove webdriver property to avoid detection
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    return driver

In [9]:
import bs4
from bs4 import BeautifulSoup


def get_hotels(driver: webdriver.Chrome, url: str) -> List[bs4.element.Tag]:
    driver.get(url)

    soup = None
    hotels=[]
    genius_modal_closed = False
    previous_hotel_count = 0
    while len(hotels) < 100:
        if not genius_modal_closed:
            time.sleep(6)
            if close_genius_modal(driver):
                genius_modal_closed = True

        load_more(driver)
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        hotels = soup.find_all('div', {'data-testid': "property-card"})
        
        if len(hotels) == previous_hotel_count:
            print("No new hotels found")
            break
        previous_hotel_count = len(hotels)
        
    return hotels[:100]

### Extract features into DataFrame

In [10]:
def extract_name(hotel: bs4.element.Tag) -> Dict[str, str]:
    return {
        'name': hotel.find('div', {'data-testid': 'title'}).text,
    }

In [14]:
import re
from lxml import etree

def extract_price(hotel: bs4.element.Tag) -> Dict[str, Optional[int]]:
    return {
        'price': hotel.find('span', {'data-testid': 'price-and-discounted-price'}).text
            }

In [15]:
def extract_review_info(hotel: bs4.element.Tag) -> Dict[str, str]:
    rating_div = hotel.find('div', {'data-testid': 'review-score'})
    if not rating_div:
        return {
            'review_score': None,
            'review_title': None,
            'number_of_reviews': None
        }
    review_divs = rating_div.find_all('div')
    
    return {
        'review_score': review_divs[0].get_text(separator=':)', strip=True).split(':)')[1],
        'review_title': review_divs[3].get_text(strip=True),
        'number_of_reviews': review_divs[4].get_text(separator=' ', strip=True).split(' ')[0]
    }


In [16]:
def extract_star_rating(hotel: bs4.element.Tag) -> Dict[str, Optional[int]]:
    star_rating_div = hotel.find('div', {'data-testid': 'rating-stars'})
    if star_rating_div:
        return {'star_rating': len(star_rating_div.find_all('svg'))}
    else:
        return {'star_rating': None}


In [17]:
import re
def extract_kms_from_centre(hotel: bs4.element.Tag) -> Dict[str, Optional[float]]:
    distance_div = hotel.find('span', {'data-testid': 'distance'})
    if  not distance_div:
        return {'kms_from_centre': None}
        
    distance = distance_div.get_text(strip=True)
    if not distance:
        return {'kms_from_centre': None}
        
    from_centre = re.search(r'([\d]+(?:\.\d+)?)\s*km\s+from\s+centre', distance)
    if not from_centre:
        return {'kms_from_centre': None}

    return {'kms_from_centre': float(from_centre.group(1))}
    

In [18]:
def extract_location_score(hotel: bs4.element.Tag) -> Dict[str, Optional[float]]:
    location_link = hotel.find('a', {'data-testid': 'secondary-review-score-link'})
    
    if not location_link:
        return {'location_score': None}
    
    location_score_match = re.search(r'Scored\s(\d+\.\d+)', location_link['aria-label'])
    
    if not location_score_match:
        return {'location_score': None}
    
    return {'location_score': float(location_score_match.group(1))}


In [19]:
def extract_sleeping_arrangements(hotel: bs4.element.Tag) -> Dict[str, Optional[str]]:
    recommended_units_div = hotel.find('div', {'data-testid': 'recommended-units'})
    
    sleeping_arrangements = {}
    
    sleeping_arrangements['room_type'] = (recommended_units_div and 
                               recommended_units_div.h4 and 
                               recommended_units_div.h4.get_text(strip=True)) or None
    
    sleeping_arrangements['bed_type'] = (recommended_units_div and 
                              recommended_units_div.ul and 
                              recommended_units_div.ul.li and 
                              recommended_units_div.ul.li.div and 
                              recommended_units_div.ul.li.div.div and 
                              recommended_units_div.ul.li.div.div.get_text(strip=True)) or None
    
    return sleeping_arrangements


In [20]:
def extract_breakfast_included(hotel: bs4.element.Tag) -> Dict[str, bool]:
    
    if 'breakfast included' in hotel.get_text(strip=True).lower():
        return {'breakfast_included': True}
    else:
        return {'breakfast_included': False}


In [21]:
def extract_payment_policy(hotel: bs4.element.Tag) -> Dict[str, str]:
    payment_policy = {}
    
    payment_policy['free_cancellation'] = True if hotel.find('span', {'data-testid': 'cancellation-policy-icon'}) else False
    payment_policy['prepayment_needed'] = False if hotel.find('span', {'data-testid': 'prepayment-policy-icon'}) else True 
    
    return payment_policy


In [22]:
def extract_hotel_info(hotel: bs4.element.Tag) -> Dict[str, Any]:
    hotel_info = {}
    
    # basic info
    extraction_functions = [
        extract_name,
        extract_price, 
        extract_review_info,
        extract_star_rating,
        extract_kms_from_centre,
        extract_location_score,
        extract_sleeping_arrangements,
        extract_breakfast_included,
        extract_payment_policy
        
    ]
    
    for func in extraction_functions:
        hotel_info.update(func(hotel))

    return hotel_info

In [22]:
import pandas as pd 

def get_hotels_dataframe(hotels: List[bs4.element.Tag]) -> pd.DataFrame:
    hotels_info = [extract_hotel_info(hotel) for hotel in hotels]
    return pd.DataFrame(hotels_info)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from datetime import datetime
import os
import math

SNAPSHOT_DATE = datetime.today().strftime('%Y-%m-%d')

number_of_cores = os.cpu_count()
max_workers = math.ceil(number_of_cores/3)


def scrape_hotels_thread(TTT, LOS):
    """Scrape hotel data using a shared WebDriver with multiple tabs."""
    print(f'scraping TTT={TTT} / 30, LOS={LOS} / 5')
    try:
        url = build_url(TTT, LOS)

        driver = init_driver(headless=True)  
        # Open a new tab
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])  # Switch to new tab
        driver.get(url)

        hotels = get_hotels(driver, url)
        hotels_df = get_hotels_dataframe(hotels)
        hotels_df['time_to_travel'] = TTT
        hotels_df['length_of_stay'] = LOS
        hotels_df['snapshot_date'] = SNAPSHOT_DATE

        driver.close()  # Close tab after scraping
        driver.switch_to.window(driver.window_handles[0])  # Switch back to main tab

        return hotels_df
    
    except Exception as e:
        print(f"Error scraping TTT={TTT}, LOS={LOS}: {e}")
        return pd.DataFrame()  # Return empty DataFrame in case of failure
    finally:
        if driver:
            driver.quit()

all_combinations = [(TTT, LOS) for TTT in range(1, 31) for LOS in range(1, 6)]

# Run threads, each using a new tab in the same browser
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    results = list(executor.map(lambda args: scrape_hotels_thread(*args), all_combinations))
    df = pd.concat(results, ignore_index=True)
    df.to_csv(f'booking_snapshot_{SNAPSHOT_DATE}.csv', index=False)